# World Airport Imagery Retrieval


In [1]:
%config IPCompleter.greedy=True
import folium
import os
from zipfile import ZipFile
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
import fiona
import pyproj
import gdal
from datetime import datetime
import pygc
from io import StringIO
import shutil
import cv2
from sklearn import cluster
from osgeo import gdal, gdal_array
from sklearn.cluster import KMeans
from PIL import Image
import scipy.misc
import json

In [2]:
missing_values = [r"\N"]
df0 = pd.read_csv(r'airport-codes_csv.csv',na_values = missing_values)
list1,list2,a,b = [],[],[],[]
for itema,itemb in zip(df0["iata_code"],df0['iso_region']):
    list1.append(itema)
    list2.append(itemb)
    
for items in df0['coordinates']:
    a.append(items.split(',')[0])
    b.append(items.split(',')[1])

In [25]:
iatalist,iatabatch= [],[]
batch_ranges = []
batch_size = 500
batch_number = 1
missing_values = [r"\N"]
#df = pd.read_excel(r'190812_CompanyRelations.xlsx', sheet_name = 'Airport Data',na_values = missing_values)
df = pd.read_csv(r'airportDB.csv')
df_all = df
#.sort_values(by = ['2018 Passengers'], ascending = False)
#t = np.invert(df['2018 Passengers'].isnull())
#df = df[t]
#df = df.sort_values(by = ["2018 Passengers"], ascending = False) 
allNames = []
for items,names in zip(df_all['IATA'],df_all['Name']):
    iatabatch.append(items)
    allNames.append(names)
    
def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

iatabatch = Remove(iatabatch)
chunk_items = len(iatabatch) - 1

for i in range(chunk_items):
    batch_ranges.append(iatabatch[i*batch_size : (i+1)*batch_size])
    
batch_ranges

for i in range(batch_size):
    try:
        iatalist.append(batch_ranges[batch_number -1][i])
    except:
        continue
iatalist,allNames[:1000]
air = dict(zip(iatalist,allNames[:1000]))

We have a database file of 6000+ airports containing their Latitude,Longitude,IATA codes etc. We use the consolidated list of interested we generated in the last block to query for the same from our Airports.dat file and prepare a new list of airport codes.

In [26]:
class Airport:    
    def __init__(self,name,city,con,iata,icao,lat,lon,alt):
        self.name = name
        self.city = city
        self.country = con
        self.iata = iata.upper() #3 letter
        self.icao = icao.upper() #4 letter
        self.lat = float(lat)
        self.lon = float(lon)
        self.alt = float(alt)
        
    def __str__(self):
        return ( str(self.iata+':'+self.name +':'+self.country))
    def __repr__(self):
        return ( str(self.iata+':'+self.name +':'+self.country))

airportPath = "airports.dat"

codes = {}
apts = []
with open(airportPath, encoding = "utf8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        apt = Airport(row[1],row[2],row[3],
                      row[4],row[5],
                      float(row[6]),float(row[7]),float(row[8]))
        codes[row[4].upper()] = apt
        apts.append(apt)
        
#print("found %s airports"%(len(apts)))
X = codes["MEL"].lon
Y = codes["MEL"].lat
iata,iatalat,iatalon,notthere = [],[],[],[]
for key in iatalist:
    if key in list1:
        iata.append(key)
        #have changed codes --> list1 & codes[key].lat/lon --> a/b[list1.index(key)]
        iatalon.append(a[list1.index(key)])
        iatalat.append(b[list1.index(key)])
    else : 
        notthere.append(key)
    

#print(iata[0],iatalat[0],iatalon[0])


len(iatalat)

#printing the entries that we have in our excel but not in our database(significantly improved results over the .dat file previously supplied)
notthere

[]

Using Python graphic library, we try to virtually construct a geo-boundary translated from our required distance (typically around 10-20kms). 

In [27]:
def latLonBoxByWandH(lat,lon,ew_width,ns_height):
    lats, lons = [], []
    #distance in m, az (in deg), lat (in deg), long (in deg)

    res = pygc.great_circle(distance=ew_width/2, azimuth=90, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']

    res = pygc.great_circle(distance=ns_height/2, azimuth=180, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ew_width, azimuth=270, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ns_height, azimuth=0, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ew_width, azimuth=90, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)
    
    return {'lats':lats,'lons':lons}


#test the function
#lengths in m
ew_width = 10000
ns_height = 10000
loc_lat,loc_lon = [], []
#loc_lat = [51.4775, 51.148056,1.359167]
#loc_lon = [-0.461389, -0.190278, 103.989444]
for i in range(0,len(iata)):
    loc_lat.append(iatalat[i])
    loc_lon.append(iatalon[i])
    #print(iata[i])

#loc_lon,loc_lat
loc_lat = np.asfarray(loc_lat,float)
loc_lon = np.asfarray(loc_lon,float)
len(loc_lat),len(loc_lon)

(500, 500)

Once we construct the spatial boundary, we then try to project in on a basemap of the globe and use Folium to try and do a sanity check if our geo-boundaries actually fall under the area of our concern, in this case airports.

In [ ]:
polygons,footprint =[], []

for lat, lon in zip(loc_lat,loc_lon):

    box = latLonBoxByWandH(lat,lon,ew_width,ns_height)
    
    polygon_geom = Polygon(zip(box['lons'], box['lats']))
    footprint.append(polygon_geom)
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
    
    polygons.append(polygon)


pyproj.Proj("+init=epsg:4326")

m = folium.Map([loc_lat[0],loc_lon[0]], zoom_start=12,tiles = "https://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}{r}.png",
attr = '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>')
for polygon in polygons:
    folium.GeoJson(polygon).add_to(m)
    
m

C:\Anaconda3\envs\fastai_v1\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Anaconda3\envs\fastai_v1\lib\site-packages\pyproj\crs\crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


In [ ]:
m.save('500_formetaArticle.html')

Making a query to Sentinel API using our boundary and specifying key parameters such as intersection, cloud cover, timeline of photo and the band of satellite.

In [7]:
user = 'demi12395' 
password = 'Sutd1234' 

api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [8]:
apiq = [] 
for i in range(0,len(polygons)):
    apiq.append(api.query(footprint[i],
                 date = ('20200112','20200914'),
                 platformname = 'Sentinel-2',
                 processinglevel = 'Level-2A',
                area_relation = ('Contains'),
                 cloudcoverpercentage = (0,20)))
for i in range(len(apiq)):
    print(iatalist[i],len(apiq[i]))

ATL 16
PEK 38
DXB 35
LAX 18
HND 12
ORD 6
LHR 16
HKG 5
PVG 21
CDG 31


Once we get a list of products (images in sentinel terminology), we try to work with the database and retrieve a consolidated list of product ID and title ID wherein the former is used as an unique identifier (to make another download query) in sentinel databse and latter gets stored as a folder in our local drive.

In [9]:
products_list, products_list_sorted,images,title,titlelist,best,bestlist =[],[], [], [], [] , [],[]
for products in apiq:
        products_list.append(api.to_geodataframe(products))
        
#Sorting the list of products within our array of locations for minimum cloudcover
for products in products_list:
    products_list_sorted.append(products.sort_values(['cloudcoverpercentage'],ascending = [False]))
    

for i in range(0,len(products_list_sorted)):
        images.append(products_list_sorted[i].head(1))

#for products in images: 
        #title.append(products.title)
        #best.append(products.uuid)
        
for items in title: 
        items[18:][:99999]
for products in images:
    for i in range(0,len(products)):
        title.append(products.title[i])
        best.append(products.uuid[i])
    bestlist.append(best)
    titlelist.append(title)
    title,best = [], []
        
#titlelist
#bestlist

C:\Anaconda3\envs\fastai_v1\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [10]:
productsDF = pd.DataFrame(images[1])

In [11]:
productsDF.head().transpose()

,6c53edc6-ebf9-4515-924c-09988d45f9c5
title,S2A_MSIL2A_20200113T031101_N0213_R075_T50TMK_2...
link,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
link_alternative,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
link_icon,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
summary,"Date: 2020-01-13T03:11:01.024Z, Instrument: MS..."
beginposition,2020-01-13 03:11:01.024000
endposition,2020-01-13 03:11:01.024000
ingestiondate,2020-01-13 12:07:11.173000
orbitnumber,23810
relativeorbitnumber,75


Now, we are trying to retrieve the datestamp of our product from the database and storing it seperately for our records.

In [12]:
datestamp,stamp = [],[]
for k in range(0,len(bestlist)):
    for i in range(0,len(bestlist[k])):
        d = products_list[k].beginposition[i].date().strftime("%Y%m%d")
        stamp.append(d)
    datestamp.append(stamp)
    stamp = []
#products_list[1]
#datestamp

In [13]:
#kth best image available
k = 0

In [14]:
prodlist = bestlist
#prodlist

In [15]:
titlelist

[['S2A_MSIL2A_20200718T161911_N0214_R040_T16SGC_20200718T203123'],
 ['S2A_MSIL2A_20200113T031101_N0213_R075_T50TMK_20200113T071306'],
 ['S2B_MSIL2A_20200304T064749_N0214_R020_T40RCN_20200304T113926'],
 ['S2B_MSIL2A_20200203T183549_N0213_R027_T11SLT_20200203T210404'],
 ['S2A_MSIL2A_20200830T012701_N0214_R074_T54SUE_20200830T041851'],
 ['S2A_MSIL2A_20200724T163901_N0214_R126_T16TDM_20200724T205753'],
 ['S2A_MSIL2A_20200406T110621_N0214_R137_T30UXC_20200406T120807'],
 ['S2B_MSIL2A_20200822T025549_N0214_R032_T49QGE_20200822T050336'],
 ['S2A_MSIL2A_20200314T023551_N0214_R089_T51RUQ_20200314T045440'],
 ['S2B_MSIL2A_20200425T104619_N0214_R051_T31UDQ_20200425T134321']]

Now we are trying to make an API call to download our product. Note that we are adding a method to extract the product and remove the parent file just to conservr memory as each product could take upto 1 GB of storage space.

In [16]:
directory  = "Products"
counter = 1
for prods,titles in zip(prodlist,titlelist):
    for items,items2 in zip(prods,titles):
        print("Downloading product " + items2 + " ... " + str(counter) + "/" + str(len(titlelist)))
        counter+=1
        api.download(items, directory_path = directory)
        with ZipFile("Products/" + items2 + '.zip', 'r') as zf:
            print("extracting  " + items2)
            zf.extractall('Products/')
            zf.close()  
        print("Unzipping complete.. Now removing the file..")
        os.remove("Products/" + items2 + '.zip')

Downloading: 100%|████████████████████████████████████████████████████████████████| 1.19G/1.19G [03:52<00:00, 5.13MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.19G/1.19G [00:03<00:00, 369MB/s]


extracting  S2A_MSIL2A_20200718T161911_N0214_R040_T16SGC_20200718T203123
Unzipping complete.. Now removing the file..


Downloading: 100%|████████████████████████████████████████████████████████████████| 1.21G/1.21G [05:24<00:00, 3.73MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.21G/1.21G [00:02<00:00, 415MB/s]


extracting  S2A_MSIL2A_20200113T031101_N0213_R075_T50TMK_20200113T071306
Unzipping complete.. Now removing the file..


Downloading: 100%|████████████████████████████████████████████████████████████████| 1.19G/1.19G [08:31<00:00, 2.32MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.19G/1.19G [00:03<00:00, 383MB/s]


...

extracting  S2B_MSIL2A_20200822T025549_N0214_R032_T49QGE_20200822T050336
Unzipping complete.. Now removing the file..


Downloading: 100%|████████████████████████████████████████████████████████████████| 1.07G/1.07G [06:06<00:00, 2.93MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.07G/1.07G [00:02<00:00, 403MB/s]


extracting  S2A_MSIL2A_20200314T023551_N0214_R089_T51RUQ_20200314T045440
Unzipping complete.. Now removing the file..


Downloading: 100%|████████████████████████████████████████████████████████████████| 1.18G/1.18G [06:35<00:00, 2.97MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.18G/1.18G [00:02<00:00, 399MB/s]


extracting  S2B_MSIL2A_20200425T104619_N0214_R051_T31UDQ_20200425T134321
Unzipping complete.. Now removing the file..


In [17]:
folder,foldername,dire,allmapsinc,allmaps,diretemp =[], [], [],[],[],[]
for titles in titlelist:
    for items in titles:
        d = os.listdir("Products/" + items + ".SAFE/" + "GRANULE")
        folder.append(d[0])
    foldername.append(folder)
    folder = []
foldername

for titles,folders in zip(titlelist,foldername):
    for items,names in zip(titles,folders):
        maindir = "Products/" + items + ".SAFE/" + "GRANULE/"  +str(names) + "/IMG_DATA" + "/R10m/" 
        allmapsinc.append(maindir)
        diretemp.append(os.listdir(maindir))
    allmaps.append(allmapsinc)
    dire.append(diretemp)
    allmapsinc,diretemp = [],[]

In [18]:
#EDITED BY SCJ
templatemp,template = [],[]
for k in range(0,len(dire)):
    for i in range(0,len(dire[0])):
        #templatemp.append(dire[k][i][-2][0:][:-4]) #SCJ removed as seems to rely on file order, doesn't work on mac
        files = [a for a in dire[k][i] if "_TCI_10m.jp2" in a]
        #print(files)
        templatemp.append(files[0][:-4])   
    template.append(templatemp)
    templatemp = []

In [19]:
#template

Dynamically retrieving the directory that we need for processing and rendering our image files.

In [20]:
allmapscompleteinc,allmapscomplete = [],[]
for k in range(0,len(prodlist)):
    for i in range(0,len(prodlist[0])):
        allmapscompleteinc.append(allmaps[k][i] + "/" + template[k][i] + ".jp2")
    allmapscomplete.append(allmapscompleteinc)
    allmapscompleteinc = []
#allmapscomplete

In [21]:
dataArr = []
for items in allmapscomplete:
    data = rio.open(items[0])
    #print(items)
    #print(data.meta['crs'])
    dataArr.append(data)
    data.close()

Opening a test image using Rasterio and checking its projection CRS and other metadata.

### Mask Satellite images

Now we try to reproject our images into EPSG system in order to average out for all the different parts of earth. This process could take a while.

In [22]:
from osgeo import gdal
filename,file = [],[]
#gdoptions = gdal.WarpOptions(options = ['dstSRS = "WGS84"','xRes = 10', "yRes = 10"," targetAlignedPixels = True"])
gdoptions = gdal.WarpOptions('dstSRS = WGS84')
for k in range(0,len(allmaps)):
    for i in range(0,len(allmaps[0])):
        input_raster = gdal.Open(allmapscomplete[k][i])
        output_raster = allmaps[k][i] + template[k][i] +".tiff"
        file.append(output_raster)
        options=['dstSRS = EPSG:4326','width = 10980', 'height = 10980']
        gdal.Warp(output_raster,input_raster,srcSRS = str(dataArr[k].meta['crs']),dstSRS = 'epsg:4326',width = str(dataArr[k].width),height = str(dataArr[k].height))
        print("Finished Warping for " + " " + str(k + 1) + " " + iatalist[k] )
    filename.append(file)
    file = []

Finished Warping for  1 ATL
Finished Warping for  2 PEK
Finished Warping for  3 DXB
Finished Warping for  4 LAX
Finished Warping for  5 HND
Finished Warping for  6 ORD
Finished Warping for  7 LHR
Finished Warping for  8 HKG
Finished Warping for  9 PVG
Finished Warping for  10 CDG


In [23]:
for items in filename:
    data = rio.open(items[0])
    #print(items)
    print(data.meta['crs'])
    data.close()

EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326


Nextly, we directly mask the image and write out the clipped photo in a seperate folder with our usual naming convention.

In [24]:
from rasterio.mask import mask
src_crs = []

geoms = []
for i in range(0,len(datestamp)):
    geoms.append(polygons[i]['geometry'])
    
for k in range(0,len(prodlist[0])):
    for i in range(0,len(prodlist)):
        with rasterio.open(filename[i][k]) as src:
                out_image, out_transform = mask(src, geoms[i], crop=True)
                out_meta = src.meta.copy()

        out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})
        try:
            with rasterio.open("Source/S" + "_" +  iatalist[i] + "_" + datestamp[i][k]  +".tiff", "w", **out_meta) as dest:
                dest.write(out_image)
            #os.remove(titlelist[k][i])
            src.close()
            dest.close()
        except:
            continue
        